## 1.Import Libraries 

In [1]:
import os
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split

## 2.Reading the Data

In [2]:
Project_dr = r"H:\Projects\Flight-Price_Prediction"
Data_dr = "Data"

In [3]:
def get_data(name):
    File_name = f"{name}.csv"
    File_path = os.path.join(Project_dr,Data_dr,File_name)
    return pd.read_csv(File_path)

In [4]:
flight_df = get_data("flight_price")
flight_df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [5]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The data set contains 10,683 rows.
- Columns `Route` and `Total_space` have a missing value each.
- The datatype are not appropriate according to data(Example- Date has a object datatype , Dep_time and Arrival Time has a object datatype this is wrong)

## 3.Preliminary Data Analysis

### 3.1 Check Datatype

In [6]:
flight_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
flight_df['Date_of_Journey'].iloc[5]

'24/06/2019'

In [8]:
flight_df['Dep_Time'].iloc[5]

'09:00'

In [9]:
flight_df['Arrival_Time'].iloc[5]

'11:25'

- Above `Date_of_Journey` , `Dep_Time` , `Arrival_Time` Feature datatype is not appropriate.
- It should be change as Date,Time.

In [10]:
flight_df['Duration'].iloc[5]

'2h 25m'

In [11]:
flight_df['Total_Stops'].iloc[5]

'non-stop'

- In `Duration` Feature I change time duration in minute format.(Example- 2h 25m = 145m)
- `Total_Stops` Feature I change this into 0 1 2 3 format.(Example- non-stop = 0 , 2 stops = 2 , 1 stops = 1)

### 3.2 Check for Duplicate

In [12]:
flight_df.duplicated().sum()

220

In [13]:
(
    flight_df
    .loc[flight_df.duplicated(keep=False)]
    .sort_values(["Airline" , "Date_of_Journey" , "Source" , "Destination"])

)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


#### Observation's

- The type of this feature `Date_of_Journey` , `Dep_Time` , `Arrival_Time` is changed to Date , Time.
- The type of this feature `Duration` , `Total_Stops` is changed to the numeric.
- In this dataset 220 duplicates.These should be remove.

## 4.Detailed Analysis

#### Airline

In [14]:
flight_df.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [15]:
flight_df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- In Airline feature some of the value are inconsistent.(Example - `Vistara` and `Vistara Premium economy` is same Airline `Vistara` )

In [21]:
(
    flight_df
    .Airline
    .str.replace(" Premium economy" , "")
    .str.replace(" Business" , "")
    .str.title()
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

#### Date_of_Journey 

In [29]:
flight_df.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [14]:
(
    pd.to_datetime(flight_df.Date_of_Journey,dayfirst=True)
)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

- Date_of_journey feature has dtype as object it's changing as Date.

### Source

In [20]:
flight_df.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [22]:
flight_df.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Dep_Time

In [15]:
flight_df.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [34]:
(
    flight_df
    .Dep_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [26]:
pd.to_datetime(flight_df.Dep_Time , format= '%H:%M').dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival Time

In [29]:
flight_df.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [22]:
(
    flight_df
    .Arrival_Time
    .loc[lambda x : x.str.contains("[^0-9:]")]
    .str.split(" " ,n=1)
    .str.get(1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### Duration 

In [40]:
(
    flight_df
    .Duration
    .loc[lambda ser: ~ser.str.contains("m") ]
    .unique()
    
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [42]:
(
    flight_df
    .Duration
    .loc[lambda ser: ~ser.str.contains("h") ]
    # .unique()
    
)

6474    5m
Name: Duration, dtype: object

In [44]:
flight_df.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [71]:
(
    flight_df
    .Duration
    .drop(index= 6474 )
    .str.split(" " , expand=True)
    .set_axis(["hour" , "minute"] ,axis =1 )
    .assign(
        hour = lambda ele : (
            ele
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minute = lambda ele : (
            ele
            .minute
            .str.replace("m" , "")
            .fillna("0")
            .astype(int)
        )
    )
    # .dtypes
    # .isna().sum()
    .sum(axis=1)
    .rename("duration_time")
    .to_frame()
    .join(flight_df.Duration.drop(index = 6474))
)

,duration_time,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


### Total_Stops

In [88]:
flight_df.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [91]:
(
    flight_df
    .Total_Stops
    .replace("non-stop" , "0")
    .str.replace(" stops?" , "" , regex=True) # In this (before ?) ? denote that the it is regular expression.
    .pipe(lambda ser :pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### Additional Information

In [96]:
flight_df.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

## 5.Cleaning Operation

In [82]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" " , expand=True)
    .set_axis(["hour" , "minute"] ,axis =1 )
    .assign(
        hour = lambda ele : (
            ele
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minute = lambda ele : (
            ele
            .minute
            .str.replace("m" , "")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    )
    

In [151]:
def clean_data(df):
    return (
        df
        .drop(index=6474)
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include='O').columns
            
        })
        .rename(str.lower , axis = 'columns')
        .assign(
            airline = lambda x : ( x
                      .airline
                      .str.replace(" Premium economy" , "")
                      .str.replace(" Business" , "")
                      .str.title()
                     ) , 
            date_of_journey = lambda x : pd.to_datetime(x.date_of_journey,dayfirst=True),
            dep_time = lambda ser : pd.to_datetime(ser.dep_time).dt.time,
            arrival_time = lambda ser : pd.to_datetime(ser.arrival_time).dt.time,
            duration = lambda ser : ser.duration.pipe(convert_to_minutes),
            total_stops = lambda num : (
                num
                .total_stops
                .replace("non-stop" , "0")
                .str.replace(" stops?" , "" , regex=True) # In this (before ?) ? denote that the it is regular expression.
                .pipe(lambda ser :pd.to_numeric(ser))
            ),
            additional_info = lambda x : x.additional_info.replace("No info","No Info")
        )
        .drop(columns = "route")
    )

In [152]:
flight_cleaned = clean_data(flight_df)
flight_cleaned

C:\Users\dhrup\AppData\Local\Temp\ipykernel_18084\1575183536.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time = lambda ser : pd.to_datetime(ser.dep_time).dt.time,
C:\Users\dhrup\AppData\Local\Temp\ipykernel_18084\1575183536.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda ser : pd.to_datetime(ser.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


## 6.Split the Data